In [11]:
# 操作 browser 的 API
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 加入行為鍊 ActionChain (在 WebDriver 中模擬滑鼠移動、點繫、拖曳、按右鍵出現選單，以及鍵盤輸入文字、按下鍵盤上的按鈕等)
from selenium.webdriver.common.action_chains import ActionChains

# 強制等待 (執行期間休息一下)
from time import sleep
import time

# 整理 json 使用的工具
import json

# 執行 command 的時候用的
import os

# 子處理程序，用來取代 os.system 的功能
import subprocess

import pandas as pd

import pprint


# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")                #不開啟實體瀏覽器背景執行
my_options.add_argument("--start-maximized")  # 最大化視窗
my_options.add_argument("--incognito")  # 開啟無痕模式
my_options.add_argument("--disable-popup-blocking")  # 禁用彈出攔截
my_options.add_argument("--disable-notifications")  # 取消 chrome 推播通知
my_options.add_argument("--lang=zh-TW")  # 設定為正體中文

# 使用 Chrome 的 WebDriver
driver = webdriver.Chrome(
    options=my_options,
    service=Service(ChromeDriverManager().install())
)

# 建立儲存文件的資料夾
folderPath = 'C:/Users/student/Desktop/期末專題/音樂情境推薦系統/music_code'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)


# 放置爬取的資料
listSong = []

In [7]:
# 走訪網址
def visit():
    driver.get('https://www.youtube.com/')


def search():
    inputTxt = driver.find_element(By.CSS_SELECTOR, 'input#search')
    inputTxt.send_keys('療傷歌單')
    sleep(2)
    inputTxt.submit()
    
    
def scroll():
    '''
    innerHeight => 瀏覽器內部的高度
    offset => 當前捲動的量(高度)
    count => 累計無效滾動次數
    limit => 最大無效滾動次數
    '''
    innerHeight = 0
    offset = 0
    count = 0
    limit = 3
    
    # 在捲動到沒有元素動態產生前，持續捲動
    while count <= limit:
        # 每次移動高度
#         offset += 1000
        offset = driver.execute_script(
            'return window.document.documentElement.scrollHeight;'
        )
        
        # 捲軸往下滑動
        driver.execute_script(f'''
            window.scrollTo({{
                top: {offset}, 
                behavior: 'smooth' 
            }});
        ''')
        
        # 強制等待，此時若有新元素生成，瀏覽器內部高度會自動增加
        sleep(2)
        
        
        # 透過執行 js 語法來取得捲動後的當前總高度
        innerHeight = driver.execute_script(
            'return window.document.documentElement.scrollHeight;'
        );
        
        # 經過計算，如果滾動距離(offset)大於等於視窗內部總高度(innerHeight)，代表已經到底了
        if offset == innerHeight:
            count += 1    
    
    
def getMainLink():
    # 建立儲存圖片、影片的資料夾
    audio = f'{folderPath}/療傷歌單'
    if not os.path.exists(audio):
        os.makedirs(audio)
        
    listSongs = driver.find_elements(By.CSS_SELECTOR, 'ytd-playlist-renderer.style-scope.ytd-item-section-renderer')
    
    for info in listSongs:
        name = info.find_element(By.CSS_SELECTOR, 'div#content > a > h3 > span#video-title')  # 取得清單名
        listName = name.get_attribute('innerText')
        
        Song = info.find_element(By.CSS_SELECTOR, 'a')  # 取得清單連結
        listLink = Song.get_attribute('href')
        
        print("=" * 30)
        print(listName)
        print(listLink)
        
        listSong.append({
            "name": listName,
            "link": listLink
        })

    # 建立歌單 json 檔
    with open(f"{folderPath}/療傷歌單.json", "w", encoding="utf-8") as file:
        file.write(json.dumps(listSong, ensure_ascii=False))
    
 


def getSubLink():
    for i in ( len(listSong) ):
        driver.get( listSong[i][link] )
        # 若是走訪網頁時，等待不到特定的元素，視為沒有資料，continue 到 for 的下一個 index 去
        try:
            title = 'ytd-playlist-panel-renderer#playlist > div.style-scope.ytd-playlist-panel-renderer > div.playlist-items.style-scope.ytd-playlist-panel-renderer > ytd-playlist-panel-video-renderer > a'
            # 等待元素
            WebDriverWait(driver, 5).until(
                EC.presence_of_element_located(
                    (By.CSS_SELECTOR, title)
                )
            )
            
            # 整理音樂清單
            list_elms = driver.find_elements(By.CSS_SELECTOR, title)
            for a in list_elms:
                listSong[i]["sub"].append({
                    "sub_title": a.get_attribute("innerText"),
                    "sub_link": parse.unquote( a.get_attribute("href") )
                })
        except TimeoutException as e:
            continue
    
    
    
    
    
def download():
    # 開啟 json 檔案
    audio = f'{folderPath}/療傷歌單'
    with open(f"{audio}/療傷歌單.json", "r", encoding='utf-8') as file:
        #取得 json 字串
        strJson = file.read()
        
    # 將 json 轉成 list (裡面是 dict 集合)
    listResult = json.loads(strJson)
    
    # 下載所有檔案
    for index, obj in enumerate(listResult):   
        print("=" * 50)
        print(f"正在下載連結: {obj['link']}")
        
        # 定義指令
        cmd = [
            './yt-dlp.exe', 
            obj['link'], 
            '-f', 'b[ext=mp3]', 
            '-o', f'{audio}/%(index)s.%(ext)s'
        ]

        # 執行指令，並取得回傳結果
        result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)

        # 將回傳結果進行解碼，顯示實際執行過程的文字輸出
        output = result.stdout
        print("下載完成，訊息如下:")
        print(output)





In [12]:
if __name__ == "__main__":
    visit()
    search()
    scroll()
    getMainLink()

失戀者的痊癒歌單】
https://www.youtube.com/watch?v=Ew4VvF0DPMc&list=PLmaQtAlowFu2DTqKKPv12gWAtLqSii7x6
必哭大哭歌單
https://www.youtube.com/watch?v=Zs8TRQ4PTYk&list=PLYPb9g4TGPFfr_p7MBD6Z4Sf0r9k3awqM
2022新歌 & 排行榜歌曲 - 中文歌曲排行榜2022
https://www.youtube.com/watch?v=T253Ss8-Qro&list=PL3oW2tjiIxvTUfDOkivqSDxrxfQccwxsN
療傷歌單
https://www.youtube.com/watch?v=xWTiOqJqkk0&list=PLVyXmu3OVygWDj96QI-eZFuPXQ_sRA8Qw
台語歌 精選.經典.老歌 [好聽.回憶](Nice song of Taiwan )
https://www.youtube.com/watch?v=h7_Z_U1UUiI&list=PLEwkLcLx23S6XDr4EWEIDLN7VSMakWGlM
療傷歌單
https://www.youtube.com/watch?v=QIIO6JvsukM&list=PLv2gp5jD61aJt_G7nRO9s4e_P3KMnOJ43
EMO period
https://www.youtube.com/watch?v=w2fkxXAMLjY&list=PLVy1epGbBHsDxbxBklbaxTLEfPVKaFto8
療傷歌單
https://www.youtube.com/watch?v=na_xv5iFt2Y&list=PLPUZN9C5ahYHEwjMJVBznK6JHdMfvj_l4
療傷歌單
https://www.youtube.com/watch?v=T4SimnaiktU&list=PL-QyP6dVn-krwX1g1KCm3hNfyFzaqvyax
療傷歌單
https://www.youtube.com/watch?v=t4E7FB-V3Z8&list=PL_b2lnIjn7ZTaRnGn1s1uQ9i21b25wGL7
療傷歌單
https://www.youtube.com/watch?